In [1]:
use Data::Dumper;
use Getopt::Long;
use FindBin qw($RealBin);
use File::Spec;

In [2]:
sub read_fasta_seq {
   my ($fh, $seq_info) = @_;

   $seq_info->{seq} = undef; # clear out previous sequence

   # put the header into place
   $seq_info->{header} = $seq_info->{next_header} if $seq_info->{next_header};

   my $file_not_empty = 0; 
   while (<$fh>) {
      $file_not_empty = 1;
      next if /^\s*$/;  # skip blank lines
      chomp;    

      if (/^>/) { # fasta header line
         my $h = $_;    
         $h =~ s/^>//;  
         if ($seq_info->{header}) {
            $seq_info->{next_header} = $h;
            return $seq_info;   
         }              
         else { # first time through only
            $seq_info->{header} = $h;
         }              
      }         
      else {    
         s/\s+//;  # remove any white space
         $seq_info->{seq} .= $_;
      }         
   }    

   if ($file_not_empty) {
      return $seq_info;
   }    
   else {
      # clean everything up
      $seq_info->{header} = $seq_info->{seq} = $seq_info->{next_header} = undef;

      return;   
   }    
}

In [3]:
my $MIN_LENGTH = 10000;
sub cluster{
    my $feature_f = shift; #feature file from probuild
    my $clusters = shift; #user-defiend number of bins. Default=0
    #my $out = "/home/gena/_temp_out";
    #`~alexl/DISTR/src/probuild/probuild --stat_fasta  $out   --seq $seq_file`;
    
    my %gc_hash;
    my @cut_off_points;
    my ($min_GC, $max_GC, $one_third, $two_third, $one_half);
    my $num_of_seq = 0;
    my $total_length = 0;
    my %header_to_cod_GC;

    open (GC, "<", $feature_f) or die "can't open $feature_f: $!\n";
    while (<GC>){
        next if ($_ !~ /^>(.*?)\t(\d+)\s+(\d+)/);
        my $header = $1;
        my $length = $2;
        my $GC = $3;
        if($header =~ /^(.*?)\t/){
            $header = $1;
        }
        $header_to_cod_GC{$header} = $GC;
        $num_of_seq ++;
        $total_length += $length;
        $gc_hash{$GC} += $length;
        #$gc_hash{$GC} ++;
    
    }
    close GC;
    
    my @sorted_GC = sort {$a<=>$b} keys %gc_hash;
    $min_GC = $sorted_GC[0];
    $max_GC = $sorted_GC[-1];
#     &Log ( "min_GC=$min_GC  max_GC=$max_GC total_seq_length=$total_length\n" );

    my $previous = 0;
    for my $key (@sorted_GC){
        $gc_hash{$key} += $previous;
    #	if($previous < $num_of_seq/3 && $gc_hash{$key} >= $num_of_seq/3){$one_third = $key};
    #	if($previous < $num_of_seq/3*2 && $gc_hash{$key} >= $num_of_seq/3*2){$two_third = $key};
    #	if($previous < $num_of_seq/2 && $gc_hash{$key} >= $num_of_seq/2){$one_half = $key};

        if($previous < $total_length/3 && $gc_hash{$key} >= $total_length/3){$one_third = $key};
        if($previous < $total_length/3*2 && $gc_hash{$key} >= $total_length/3*2){$two_third = $key};
        if($previous < $total_length/2 && $gc_hash{$key} >= $total_length/2){$one_half = $key};
        $previous = $gc_hash{$key};
    }
#         &Log ("($one_third)->($gc_hash{$one_third})\n");
#         &Log ("($one_half)->($gc_hash{$one_half})\n");
#         &Log ("($two_third)->($gc_hash{$two_third})\n");

    if($clusters == 0){
        #cluster number is not specified by user
        #automatically choose cluster number.
        if( $two_third - $one_third > 3){
            $clusters = 3;
        }
        else {
            $clusters = 1;
        }
    }
    if($clusters == 3){
        if($two_third - $one_third < 1 || $max_GC - $two_third < 1 || $one_third - $min_GC < 1){
#             &Log( "Total number of sequences is not enough for training in 3 clusters!\n" );
            $clusters = 1;
        }
        else{
            if($gc_hash{$one_third} > $MIN_LENGTH){
                push @cut_off_points, ($min_GC,$one_third,$two_third,$max_GC);}
            else{
#                 &Log( "Total length of sequences is not enough for training in 3 clusters!\n" );
                $clusters = 2;
            }
        }
    }
    if($clusters == 2){
        if($gc_hash{$one_half} > $MIN_LENGTH){
            push @cut_off_points, ($min_GC,$one_half,$max_GC);}
        else{
#             &Log( "Total length of sequences is not enough for training in 2 clusters!\n" );
            $clusters = 1;
        }
    }

    if($clusters == 1){
        push @cut_off_points, ($min_GC, $max_GC);
    }
    #print $clusters," ",join(" ", @cut_off_points);
    return ($clusters,\@cut_off_points,\%header_to_cod_GC);
}

10000

In [4]:
sub getGC {
        my ($seq)  = @_;
        chomp($seq);
        my $gc     = 0;
        my $acgt = 0;
        my $length = length($seq);
        $gc = () =  $seq =~ /g|c/ig; 
        $acgt = () = $seq =~ /a|c|g|t/ig;
        if ( $length > 0 ) {
                if($acgt != $length){
                        print STDERR "letters that are not ACGT!\n";
                        return ( "0_noncanonical_letter") if $acgt eq 0;
                        return ( ($gc/$acgt*100)."_noncanonical_letter");
                }
                return ($gc/$length*100);
        }
        else {
            return -1;
                #return "It's an empty sequence";
        }
}

In [5]:
my $gc_out = "pygmst/pygmst/initial.meta.lst.feature";
my $bins = 2;
my ($bin_num, $cutoffs, $seq_GC) = cluster($gc_out, $bins);

2ARRAY(0x4d01208)HASH(0x4d013b8)


In [6]:
$seq_GC

HASH(0x4d013b8)


In [7]:
my $seqfile = "pygmst/pygmst/test.fa"

pygmst/pygmst/test.fa


In [8]:
my $FA;
open($FA, $seqfile) or die "can't open $seqfile: $!\n";
my %read;

In [9]:
my @names = keys $seq_GC;

In [14]:
print Dumper(@names);

$VAR1 = '83_460_708_378_30.9:NM_001020404 hypothetical_protein';
$VAR2 = '914_2659_2772_1746_37.9:NM_001023290 beta_fructofuranosidase';
$VAR3 = '89_1498_2120_1410_37.9:NM_001020181 septin_Spn1';
$VAR4 = '1_1209_1597_1209_35.2:NM_001021605 mRNA_guanylyltransferase_Ceg1';
$VAR5 = '1215_4061_4061_2847_38.1:NM_001021915 cell_surface_agglutination_protein_Map4';
$VAR6 = '930_1967_2588_1038_35.2:NM_001020108 Golgi_GDP_mannose_transporter_Vrg4__predicted_';
$VAR7 = '1342_1806_1967_465_37.6:NM_001018561 superoxide_dismutase_Sod1';
$VAR8 = '104_586_657_483_35.6:NM_001021018 DUF1772_family_protein';
$VAR9 = '703_1467_2244_765_42.1:NM_001022865 switch_activating_protein_Sap1';
$VAR10 = '128_1798_1936_1671_40.7:NM_001021858 chaperonin_containing_T_complex_alpha_subunit_Cct1';
$VAR11 = '61_489_2633_429_36.9:NM_001019243 conserved_protein__UPF0047_family';
$VAR12 = '320_3037_3199_2718_35.5:NM_001022566 phosphatidylethanolamine_N_methyltransferase_Cho2';
$VAR13 = '128_1735_1846_1608_36.0:NM_00102206

1


In [15]:
print Dumper($seq_GC);

$VAR1 = {
          '83_460_708_378_30.9:NM_001020404 hypothetical_protein' => '31',
          '914_2659_2772_1746_37.9:NM_001023290 beta_fructofuranosidase' => '38',
          '89_1498_2120_1410_37.9:NM_001020181 septin_Spn1' => '38',
          '1_1209_1597_1209_35.2:NM_001021605 mRNA_guanylyltransferase_Ceg1' => '35',
          '1215_4061_4061_2847_38.1:NM_001021915 cell_surface_agglutination_protein_Map4' => '38',
          '930_1967_2588_1038_35.2:NM_001020108 Golgi_GDP_mannose_transporter_Vrg4__predicted_' => '35',
          '1342_1806_1967_465_37.6:NM_001018561 superoxide_dismutase_Sod1' => '38',
          '104_586_657_483_35.6:NM_001021018 DUF1772_family_protein' => '36',
          '703_1467_2244_765_42.1:NM_001022865 switch_activating_protein_Sap1' => '42',
          '128_1798_1936_1671_40.7:NM_001021858 chaperonin_containing_T_complex_alpha_subunit_Cct1' => '41',
          '61_489_2633_429_36.9:NM_001019243 conserved_protein__UPF0047_family' => '37',
          '320_3037_3199_2

1


In [16]:
read_fasta_seq($FA, \%read)

HASH(0x4c76888)


In [19]:
print ($read{header});

98_1132_1457_1035_31.3:NM_001020461 hypothetical_protein

1


In [139]:
Dumper($read{header});

$VAR1 = undef;



In [149]:
while (read_fasta_seq($FA, \%read)) {
    
        if(!exists  $seq_GC->{$read{header}}){ #no coding region in the sequence
            print $seq_GC;
            $seq_GC->{$read{header}} = getGC($read{seq});
        }
        print ">$read{header}\t[gc=$seq_GC->{$read{header}}]\n$read{seq}\n";
    }

In [147]:
Dumper(\$seq_GC);

In [148]:
print NEWINPUT ">$read{header}\t[gc=$seq_GC->{$read{header}}]\n$read{seq}\n";

In [16]:
my $alpha = read_fasta_seq($FA, \%read);

HASH(0x530fdc8)


In [150]:
close FA

In [4]:
`pwd`

/opt/jupyter



In [6]:
`ls pygmst/pygmst`

README.GeneMarkS-T
__pycache__
gm_key_64
gmst.pl
initial.meta.lst.feature
logger.py
models
pygmst.py
test.fa
test.fa.fai
test.py
utilities



In [8]:
my $FA = "pygmst/pygmst/test.fa";

pygmst/pygmst/test.fa


In [ ]:
open($FA, $seqfile)

In [130]:
my $alpha = {};

HASH(0x5f44130)


In [136]:
$alpha->{"bob"} = "bob";

bob


In [137]:
Dumper($alpha);

$VAR1 = {
          'bob' => 'bob'
        };



In [2]:
my $i = 1;
"bob".$i."carl";

bob1carl

In [7]:
`pwd`

/opt/jupyter/pygmst/pygmst/testfiles


In [8]:
my $build = "../genemark/probuild --par ../genemark/par_1.default";

../genemark/probuild --par ../genemark/par_1.default

In [5]:
sub train{
    my $input_seq = shift;
    #------------------------------------------------
    # prepare sequence
    
    &RunSystem( "$build --clean_join $seq --seq $input_seq --log $logfile", "prepare sequence\n" );
    
    push @list_of_temp, $seq;

    #------------------------------------------------
    # tmp solution: get sequence size, get minimum sequence size from --par <file>
    # compare, skip iterations if short

    my $sequence_size = -s $seq;
    $command = "grep MIN_SEQ_SIZE $par";
    my $minimum_sequence_size = `$command`;
    $minimum_sequence_size =~ s/\s*--MIN_SEQ_SIZE\s+//;

    $do_iterations = 1;

    if ( $sequence_size < $minimum_sequence_size )
    {
    &RunSystem( "$build --clean_join $seq --seq $input_seq --log $logfile --MIN_CONTIG_SIZE 0 --GAP_FILLER ", "prepare sequence\n" );
    $do_iterations = 0;
    }

    &Log( "do_iterations = $do_iterations\n" );

    #------------------------------------------------
    # run initial prediction
    $itr = 0;
    $next = &GetNameForNext( $itr );
    &RunSystem( "$hmm  $seq  -m $meta_model  -o $next", "run initial prediction\n" );
    push @list_of_temp, $next;

    #------------------------------------------------
    # enter iterations loop

    &Log( "entering iteration loop\n" );

    while( $do_iterations )
    {
      $itr++;
      $mod  = GetNameForMod( $itr );

      if ( $motif && !($fixmotif) )
      {
          $start_seq = $start_prefix . $itr;
          $gibbs_out = $gibbs_prefix . $itr;
      }

      $command = "$build --mkmod $mod --seq $seq --geneset $next --ORDM $order --order_non $order_non --revcomp_non 1";

      if ( $motif && !$fixmotif )
      { $command .= " --pre_start $start_seq --PRE_START_WIDTH $prestart"; }
      elsif ( $motif && $fixmotif )
      { $command .= " --fixmotif --PRE_START_WIDTH $prestart --width $width --log $logfile"; }

      &RunSystem( $command, "build model: $mod for iteration: $itr\n" );
      push @list_of_temp, $mod;

      if ( $motif && !$fixmotif )
      {
          if ( $gibbs_version == 1 )
          {
              &RunSystem( "$gibbs $start_seq $width -n > $gibbs_out", "run gibbs sampler\n" );
          }
          elsif ( $gibbs_version == 3 )
          {
              #&RunSystem( "$gibbs3 $start_seq $width -o $gibbs_out -F -Z  -n -r -S 20  -y -x -m  -i 2000 -w 0.01", "run gibbs3 sampler\n" );
              &RunSystem( "$gibbs3 $start_seq $width -o $gibbs_out -F -Z  -n -r -y -x -m -s 1 -w 0.01", "run gibbs3 sampler\n" );
          }
      
          push @list_of_temp, $start_seq;

          &RunSystem( "$build --gibbs $gibbs_out --mod $mod --seq $start_seq --log $logfile", "make prestart model\n" );
          push @list_of_temp, $gibbs_out;
      }

      $prev = $next;
      $next = &GetNameForNext( $itr );

      $command = "$hmm  $seq  -m $mod  -o $next";
      if ( $motif )
          { $command .= " -r"; }

      &RunSystem( $command, "prediction, iteration: $itr\n" );
      push @list_of_temp, $next;

      $command = "$build --compare --source $next --target $prev";
      &Log( "compare:\n" . $command . "\n" );

      $diff = `$command`;
      chomp( $diff );
      &Log( "compare $prev and $next: $diff\n" );

      if ( $diff >= $identity )
          { &Log( "Stopped iterations on identity: $diff\n" ); last; }
      if ( $itr == $maxitr )
          { &Log( "Stopped iterations on maximum number: $maxitr\n" ); last; }
    }
    #------------------------------------------------
    # create ouput
    
    
    if ( $do_iterations )
    {
        &RunSystem( "cp $mod $input_seq.mod", "create: $input_seq.mod\n" );
        return $input_seq.".mod";
        
    }
    else
    { 
    #	&RunSystem( "cp $imod $input_seq.mod", "create: $input_seq.mod\n" );
    #	return $meta_model;
    }

}